0) prepare date

1) model 정의 (input, output)
2) loss & optimizer
3) Training Loop
   - forward pass
   - loss
   - backward pass
   - update
   - zero grad
   - print results

In [11]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

'''0) prepare data'''
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(n_samples, n_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234) # what is random state?

# scale
sc = StandardScaler() # mean = 0 & unit variance # 왜 linear regression 에서는 안하고 logistic 에서는 하나? -> activation function 때문에 그런 것 같다. 
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# numpy to torch
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# y reshape
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


'''1) model - input, output'''
# f = wx+b, sigmois at the end
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
model = LogisticRegression(n_features, 1)

'''2) loss & optimizer'''
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''3) Training loop'''
num_epoch = 100
for epoch in range(num_epoch):
    # forward and loss
    y_pred = model(X_train)
    loss = criterion(y_pred,y_train)

    # backward
    loss.backward()

    # update
    optimizer.step()

    # zero
    optimizer.zero_grad()

    # print
    if (epoch+1) % 10 == 0:
        print(f"epoch {epoch+1} : loss = {loss.item():.4f}, {[params for params in model.parameters()]}") 

# accuracy
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

569 30
epoch 10 : loss = 0.5600, [Parameter containing:
tensor([[-0.0637,  0.0759,  0.0632, -0.1740, -0.1214, -0.1903,  0.0229, -0.1072,
          0.1093, -0.1329, -0.1329,  0.0647, -0.0206,  0.1274, -0.0673,  0.0606,
          0.0364,  0.1155,  0.1783,  0.0982,  0.0439, -0.1398, -0.2099,  0.0245,
          0.0578,  0.0679,  0.0950, -0.0945, -0.1111,  0.0323]],
       requires_grad=True), Parameter containing:
tensor([-0.0032], requires_grad=True)]
epoch 20 : loss = 0.4575, [Parameter containing:
tensor([[-0.0863,  0.0607,  0.0401, -0.1959, -0.1329, -0.2111, -0.0014, -0.1325,
          0.0966, -0.1346, -0.1521,  0.0622, -0.0397,  0.1089, -0.0680,  0.0470,
          0.0247,  0.0990,  0.1752,  0.0911,  0.0198, -0.1556, -0.2344,  0.0017,
          0.0438,  0.0474,  0.0718, -0.1207, -0.1259,  0.0194]],
       requires_grad=True), Parameter containing:
tensor([0.0097], requires_grad=True)]
epoch 30 : loss = 0.3934, [Parameter containing:
tensor([[-0.1044,  0.0479,  0.0215, -0.2136, -0.1415,